In [ ]:
# Imports
import pandas as pd
import numpy as np

stats_df_coaches = pd.read_csv('dataset/processed/coaches_processed.csv')
df_awards = pd.read_csv('dataset/processed/awards_processed.csv')

stats_df_coaches = stats_df_coaches[stats_df_coaches['stint'] <= 1].drop("stint", axis=1)

stats_df_coaches["Win Rate"] = np.where(
    (stats_df_coaches['post_wins'] == 0) & (stats_df_coaches['post_losses'] == 0),
    stats_df_coaches['won'] / (stats_df_coaches['won'] + stats_df_coaches['lost']),
    ((stats_df_coaches['won'] + stats_df_coaches['post_wins']) / (stats_df_coaches['won'] + stats_df_coaches['lost'] + stats_df_coaches['post_wins'] + stats_df_coaches['post_losses']))
)

stats_df_coaches['Cumulative Win Rate'] = 0.0
def calculate_cumulative_win_rate(group):
    group.loc[group.index[0], 'Cumulative Win Rate'] = group.loc[group.index[0], 'Win Rate']
    for i in range(1, len(group)):
        group.loc[group.index[i], 'Cumulative Win Rate'] = (
            group.loc[group.index[i], 'Win Rate'] * 0.5 + group.loc[group.index[i-1], 'Cumulative Win Rate'] * 0.5
        )
    group['Cumulative Win Rate'] = group['Cumulative Win Rate'].shift(1)
    return group

# Apply the function to each group of coachID
stats_df_coaches = stats_df_coaches.groupby('coachID').apply(calculate_cumulative_win_rate).reset_index(drop=True)
relevant_awards = df_awards[df_awards['award'] == "Coach of the Year"]
relevant_awards = relevant_awards.rename(columns={'playerID': 'coachID'})
print(relevant_awards)
award_counts = (
    relevant_awards
    .groupby(['coachID', 'year'])
    .size()
    .rename('awards')
    .reset_index()
)
stats_df_coaches = stats_df_coaches.merge(
    award_counts, 
    how='left', 
    on=['coachID', 'year'], 
)
stats_df_coaches['awards'] = stats_df_coaches['awards'].fillna(0).astype(int)
stats_df_coaches['awards'] = stats_df_coaches['awards'].shift(1)
stats_df_coaches['Cumulative Win Rate'] = stats_df_coaches['Cumulative Win Rate'] + stats_df_coaches['awards']*0.005
stats_df_coaches.drop(columns=['won', 'lost', 'post_wins', 'post_losses'], inplace=True) 

df_teams = pd.read_csv('dataset/processed/teams_test.csv')
stats_df_coaches['Cumulative Win Rate'] = stats_df_coaches['Cumulative Win Rate'].fillna(stats_df_coaches['Cumulative Win Rate'].mean())
stats_df_coaches= stats_df_coaches.drop_duplicates(subset=['tmID', 'year'], keep='first')
df_teams = pd.merge(df_teams, stats_df_coaches, on=['tmID', 'year'], how='left')
df_teams['Cumulative Win Rate'] = df_teams.groupby('tmID', group_keys=False).apply(lambda group: group.sort_values("year")['Cumulative Win Rate'].shift(-1))
df_teams['Cumulative Win Rate'] = df_teams['Cumulative Win Rate'].fillna(df_teams['Cumulative Win Rate'].mean())


df_teams.to_csv('dataset/processed/teams_test.csv', index=False)
print(stats_df_coaches[stats_df_coaches['year'] == 11])
print(stats_df_coaches.isna().sum())


       coachID              award  year
8   coopemi01w  Coach of the Year     1
9   hugheda99w  Coach of the Year     2
10  stanlma99w  Coach of the Year     3
11  laimbbi01w  Coach of the Year     4
12  mcconsu01w  Coach of the Year     5
13  whisejo99w  Coach of the Year     6
14  thibami99w  Coach of the Year     7
15  hugheda99w  Coach of the Year     8
16  thibami99w  Coach of the Year     9
17  meadoma99w  Coach of the Year    10
        coachID  year tmID  Win Rate  Cumulative Win Rate  awards
14   aglerbr99w    11  SEA       NaN             0.531397     0.0
31   chatmda99w    11  CHI       NaN             0.510222     0.0
63   dunnli99wc    11  IND       NaN             0.533553     0.0
70   gaineco01w    11  PHO       NaN             0.568627     0.0
72   gilloje01w    11  LAS       NaN             0.411765     0.0
89   hugheda99w    11  SAS       NaN             0.494516     0.0
95   laceytr99w    11  WAS       NaN             0.305147     0.0
113  meadoma99w    11  ATL      

C:\Users\JoaoCoutinho\AppData\Local\Temp\ipykernel_28236\3761302155.py:27: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  stats_df_coaches = stats_df_coaches.groupby('coachID').apply(calculate_cumulative_win_rate).reset_index(drop=True)
C:\Users\JoaoCoutinho\AppData\Local\Temp\ipykernel_28236\3761302155.py:52: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_teams['Cumulative Win Rate'] = df_teams.groupby('t